# COMP0124: Multi-Agent Artificial Intelligence

# Group project: Real-time bidding auctions

**Group #7: Oliviero Balbinetti, Mauricio Caballero, Paul Melkert**

Importing libraries.

In [51]:
import os
import sys
import math
import random
import numpy as np
import pandas as pd

from glob import glob

Defining general functions.

In [4]:
#Constant bidding strategy.
def constant_bidding(bid):
    return bid

#Random bidding strategy.
def random_bidding(lower, upper):
    return random.randint(lower, upper+1)

#Linear bidding strategy.
def linear_bidding(base_bid, pCTR, Init_CTR):
    return int(base_bid*pCTR/Init_CTR)

#Winning criterion #1.
def win_auction_1(bid, payprice):
    return bid > payprice

#Winning criterion #2.
def win_auction_2(bid, payprice, Others):
    return ((bid > payprice) and (bid > max(Others)))

#Bidding strategy parameters.
def Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices, 
                     Pred_CTRs, prm, key):
    
    empCTR = []
    empCPM = []
    Bids, Cost, Imps, Clicks = 0, 0, 0, 0
    
    #Looping on bid requests.
    for item in range(0, len(Clicks_Prices)):
        bid = 0
        Bids += 1
        
        pCTR = Pred_CTRs[item]
        empCTR.append(pCTR)
        
        #Costant strategy.
        if key == 'Constant':
            Criterion = 1
            bid = constant_bidding(prm)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
            
        #Random strategy.
        elif key == 'Random':
            Criterion = 1
            bid = random_bidding(lower, upper)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
            
        #Linear strategy.
        elif key == 'Linear':
            Criterion = 1
            bid = linear_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
            
        else:
            print('[Strategy error]: Wrong bidding strategy name!')
            sys.exit(-1)
        
        #Computing bid outcome.
        if Win:
            Imps += 1
            Cost += Clicks_Prices[item][1]
            Clicks += Clicks_Prices[item][0]
            
            if Criterion == 1:
                empCPM.append(Clicks_Prices[item][1])
            else: empCPM.append(max(Clicks_Prices[item][1],max(Others)))
            
        else:
            empCPM.append(0)
        
        #Stopping condition.
        if Cost > Budget:
            break
    
    avgCTR = np.mean(empCTR)
    avgCPM = np.mean(empCPM)
    avgCPC = avgCPM/1000
    
    Output =  key + ',' + str(prm) + ',' + str(Budget) + ',' + str(Cost) + ','\
              + str(Bids) + ',' + str(Imps) + ',' + str(Clicks) + ','\
              + str(avgCTR) + ',' + str(avgCPM) + ',' + str(avgCPC)
    
    return Output

#Bidding strategy simulation.
def Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices, Pred_CTRs, 
                     file, Dict_alg, key):
    parameters = Dict_alg[key]
    
    for prm in parameters:
        output = Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, 
                                      Clicks_Prices, Pred_CTRs, prm, key)
        file.write(output + '\n')

Importing predicted CTR data.

In [ ]:
#Defining directory.
Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data/Predicted CTR'

#Importing data.
os.chdir(Path)
Datasets_CTR = {}

for Filename in glob('*.csv'):
    Datasets_CTR[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Test_CTR = Datasets_CTR['Test']
Train_CTR = Datasets_CTR['Train']
Validation_CTR = Datasets_CTR['Validation']

Importing feature engineering data.

In [3]:
#Defining directory.
Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data/Feature engineering'

#Importing data.
os.chdir(Path)
Datasets_features = {}

for Filename in glob('*.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Test_ftr = Datasets_features['Test']
Train_ftr = Datasets_features['Train']
Validation_ftr = Datasets_features['Validation']

Training bidding strategies.

In [ ]:
#Training bidding strategies.
Budget = 6250
random.seed(10)

Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data/Parameter optimization'

#Defining general variables.
Pred_CTRs = []
Clicks_Prices = []

#Initialising parameters.
Prm_cost = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                           np.arange(100, 301, 10)), axis=0)
Prm_rand = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                           np.arange(100, 501, 10)), axis=0)
Prm_lin = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                          np.arange(100, 400, 10), np.arange(400, 800, 50)),
                          axis=0)

Dict_alg = {'Constant': Prm_cost, 'Random': Prm_rand, 'Linear': Prm_lin}

#Initialising values.
Init_CTR = sum(Train_ftr['click'])/len(Train_ftr)
Init_eCPC = sum(Train_ftr['payprice'])/(sum(Train_ftr['click'])*1000)

Pred_CTRs = list(Train_CTR['pCTR'].values)
Clicks_Prices = list(zip(Train_ftr['click'], Train_ftr['payprice']))

#Training and saving results.
os.chdir(Path)

print('Training procedures:\n')
header = 'algorithm,parameter,budget,spend,bids,imps,click,avgCTR,avgCPM,avgCPC'

for key in Dict_alg.keys():
    Filename = key + '.csv'
    
    with open(Filename, 'w') as file:
        file.write(header + '\n')
        Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices, 
                         Pred_CTRs, file, Dict_alg, key)
        
    print('• [File %s]: Process completed!\n' %Filename)

Choosing best model parameters.

In [ ]:
#Choosing best models parameters.
Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
       'Data/Parameter optimization'

#Importing data.
os.chdir(Path)
Datasets_parameter = {}

for Filename in glob('*.csv'):
    Datasets_parameter[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Linear = Datasets_parameter['Linear']
Random = Datasets_parameter['Random']
Constant = Datasets_parameter['Constant']

#Objective function.
for key in Datasets_parameter.keys():
    data = Datasets_parameter[key]
    
    df = data[data['spend'] >= Budget]
    

Evaluating bidding strategies.

In [ ]:
Repeat the simulation based on best parameters and development set.